In [5]:
import scipy.io as sio

mat_path = "Farsiu_Ophthalmology_2013_Control_Subject_1001.mat"
data = sio.loadmat(mat_path)

print(data.keys())
for k, v in data.items():
    if isinstance(v, np.ndarray):
        print(k, v.shape)

dict_keys(['__header__', '__version__', '__globals__', 'images', 'layerMaps', 'Age'])
images (512, 1000, 100)
layerMaps (100, 1000, 3)
Age (1, 1)


In [18]:
# running for one failed file. downloaded again
# binary masks for cnn + graph cut (single file version)

import os
import numpy as np
import scipy.io
from PIL import Image
from skimage import img_as_ubyte

# === INPUTS: change these ===
mat_path = "Farsiu_Ophthalmology_2013_AMD_Subject_1257.mat"  
out_dir = "AMD_masks_new"  # output folder
os.makedirs(out_dir, exist_ok=True)

# ---- helper functions ----
def interp_nan(arr):
    """1D linear interpolation for NaN values in boundary arrays"""
    idx = np.arange(arr.size)
    good = np.isfinite(arr)
    if good.sum() == 0:
        return np.zeros_like(arr)
    return np.interp(idx, idx[good], arr[good])

def boundaries_to_mask(boundaries, H):
    """Convert top & bottom boundaries into a binary mask (retina=255, background=0)"""
    W, n_bounds = boundaries.shape
    mask = np.zeros((H, W), dtype=np.uint8)
    top = np.round(boundaries[:, 0]).astype(int)
    bottom = np.round(boundaries[:, -1]).astype(int)
    for x in range(W):
        y1 = np.clip(top[x], 0, H-1)
        y2 = np.clip(bottom[x], 0, H-1)
        if y2 <= y1:  # fix inverted cases
            y1, y2 = min(y1, y2), max(y1, y2)
        mask[y1:y2+1, x] = 255
    return mask

def process_mat(mat_path, out_dir):
    try:
        mat = scipy.io.loadmat(mat_path, squeeze_me=True, struct_as_record=False)
    except Exception as e:
        print(f"❌ Failed to read {mat_path}: {e}")
        return False  # failed

    if "images" not in mat or "layerMaps" not in mat:
        print(f"⚠️ Skipping {mat_path}, keys not found")
        return False  # skipped

    images = mat["images"]       # (H, W, N_slices)
    layerMaps = mat["layerMaps"] # (N_slices, W, 3)
    H, W, N = images.shape
    subj_name = os.path.splitext(os.path.basename(mat_path))[0]

    for s in range(N):
        img = images[:, :, s]
        b = layerMaps[s]  # (W, 3)

        if not np.isfinite(b).any():
            continue  # skip empty slice

        # interpolate NaNs
        for j in range(b.shape[1]):
            if not np.isfinite(b[:, j]).all():
                b[:, j] = interp_nan(b[:, j])

        # normalize image
        img_norm = img_as_ubyte((img - img.min()) / (img.max() - img.min() + 1e-8))

        # save image
        base = f"{subj_name}_slice_{s:03d}"
        img_out = os.path.join(out_dir, base + "_img.png")
        Image.fromarray(img_norm).save(img_out)

        # save mask
        mask = boundaries_to_mask(b, H)
        mask_out = os.path.join(out_dir, base + "_mask.png")
        Image.fromarray(mask).save(mask_out)

        # save boundaries
        np.save(os.path.join(out_dir, base + "_bounds.npy"), b.astype(np.float32))

    print(f"✅ Processed {mat_path}")
    return True


# ---- run for a single file ----
print(f"Processing single file: {mat_path}")
ok = process_mat(mat_path, out_dir)

if ok:
    print("🎉 Done! Outputs saved in:", out_dir)
else:
    print("⚠️ Failed to process file.")

Processing single file: Farsiu_Ophthalmology_2013_AMD_Subject_1257.mat
✅ Processed Farsiu_Ophthalmology_2013_AMD_Subject_1257.mat
🎉 Done! Outputs saved in: AMD_masks_new


In [1]:
import os, glob
import numpy as np
import scipy.io
from PIL import Image
from skimage import img_as_ubyte

# === INPUTS ===
mat_dir = "Normal"       # folder with .mat files
out_dir = "Multiclass_normal"  # output folder for multiclass masks
os.makedirs(out_dir, exist_ok=True)

# ---- helper functions ----
def interp_nan(arr):
    """1D linear interpolation for NaN values in boundary arrays"""
    idx = np.arange(arr.size)
    good = np.isfinite(arr)
    if good.sum() == 0:
        return np.zeros_like(arr)
    return np.interp(idx, idx[good], arr[good])

def boundaries_to_multiclass_mask(boundaries, H):
    """
    Convert boundaries into a multi-class mask.
    boundaries: shape (W, n_bounds)
    mask values: 0=background, 1..n_classes = layer regions
    """
    W, n_bounds = boundaries.shape
    mask = np.zeros((H, W), dtype=np.uint8)

    # round and clip each boundary
    b = np.zeros_like(boundaries, dtype=int)
    for j in range(n_bounds):
        b[:, j] = np.clip(np.round(boundaries[:, j]).astype(int), 0, H-1)

    # fill regions between boundaries with increasing class index
    for x in range(W):
        y_prev = 0
        for j in range(n_bounds):
            y_curr = b[x, j]
            if y_curr < y_prev:  # ensure correct order
                y_curr = y_prev
            mask[y_prev:y_curr, x] = j + 1   # class labels start at 1
            y_prev = y_curr
        mask[y_prev:H, x] = n_bounds + 1     # last region = last class

    return mask

def process_mat(mat_path, out_dir):
    mat = scipy.io.loadmat(mat_path, squeeze_me=True, struct_as_record=False)
    if "images" not in mat or "layerMaps" not in mat:
        print(f"Skipping {mat_path}, keys not found")
        return
    
    images = mat["images"]       # (H, W, N_slices)
    layerMaps = mat["layerMaps"] # (N_slices, W, 3)
    H, W, N = images.shape
    subj_name = os.path.splitext(os.path.basename(mat_path))[0]

    for s in range(N):
        img = images[:,:,s]
        b = layerMaps[s]  # (W, 3)

        if not np.isfinite(b).any():
            continue  # skip empty slice

        # interpolate NaNs in each boundary column
        for j in range(b.shape[1]):
            if not np.isfinite(b[:,j]).all():
                b[:,j] = interp_nan(b[:,j])

        # normalize image to 0..255
        img_norm = img_as_ubyte((img - img.min())/(img.max()-img.min() + 1e-8))

        # save image
        base = f"{subj_name}_slice_{s:03d}"
        img_out = os.path.join(out_dir, base + "_img.png")
        Image.fromarray(img_norm).save(img_out)

        # save multi-class mask
        mask = boundaries_to_multiclass_mask(b, H)

        # --- scale for visualization (each class has unique gray intensity) ---
        mask_vis = (mask * (255 // mask.max())).astype(np.uint8)

        mask_out = os.path.join(out_dir, base + "_mask.png")
        Image.fromarray(mask_vis).save(mask_out)

        # save raw mask (true class labels, useful for training)
        np.save(os.path.join(out_dir, base + "_mask.npy"), mask.astype(np.uint8))

        # save raw boundary arrays for metrics
        np.save(os.path.join(out_dir, base + "_bounds.npy"), b.astype(np.float32))

    print(f"Processed {mat_path}")

# ---- run on all .mat files ----
mat_files = glob.glob(os.path.join(mat_dir, "*.mat"))
print(f"Found {len(mat_files)} .mat files")

for m in mat_files:
    process_mat(m, out_dir)

print("✅ All done! Multi-class processed data saved in:", out_dir)

Found 115 .mat files
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1007.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1013.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1012.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1006.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1038.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1010.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1004.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1005.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1011.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1039.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1015.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1001.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1029.mat
Processed Normal/Farsiu_Ophthalmology_2013_Control_Subject_1028.mat
Processed Normal/Farsiu_Oph

In [ ]:
import os, glob
import numpy as np
import scipy.io
from PIL import Image
from skimage import img_as_ubyte

# === INPUTS ===
mat_dir = "AMD"       # folder with .mat files
out_dir = "Multiclass_AMD"  # output folder for multiclass masks
os.makedirs(out_dir, exist_ok=True)

# ---- helper functions ----
def interp_nan(arr):
    """1D linear interpolation for NaN values in boundary arrays"""
    idx = np.arange(arr.size)
    good = np.isfinite(arr)
    if good.sum() == 0:
        return np.zeros_like(arr)
    return np.interp(idx, idx[good], arr[good])

def boundaries_to_multiclass_mask(boundaries, H):
    """
    Convert boundaries into a multi-class mask.
    boundaries: shape (W, n_bounds)
    mask values: 0=background, 1..n_classes = layer regions
    """
    W, n_bounds = boundaries.shape
    mask = np.zeros((H, W), dtype=np.uint8)

    # round and clip each boundary
    b = np.zeros_like(boundaries, dtype=int)
    for j in range(n_bounds):
        b[:, j] = np.clip(np.round(boundaries[:, j]).astype(int), 0, H-1)

    # fill regions between boundaries with increasing class index
    for x in range(W):
        y_prev = 0
        for j in range(n_bounds):
            y_curr = b[x, j]
            if y_curr < y_prev:  # ensure correct order
                y_curr = y_prev
            mask[y_prev:y_curr, x] = j + 1   # class labels start at 1
            y_prev = y_curr
        mask[y_prev:H, x] = n_bounds + 1     # last region = last class

    return mask

def process_mat(mat_path, out_dir):
    mat = scipy.io.loadmat(mat_path, squeeze_me=True, struct_as_record=False)
    if "images" not in mat or "layerMaps" not in mat:
        print(f"Skipping {mat_path}, keys not found")
        return
    
    images = mat["images"]       # (H, W, N_slices)
    layerMaps = mat["layerMaps"] # (N_slices, W, 3)
    H, W, N = images.shape
    subj_name = os.path.splitext(os.path.basename(mat_path))[0]

    for s in range(N):
        img = images[:,:,s]
        b = layerMaps[s]  # (W, 3)

        if not np.isfinite(b).any():
            continue  # skip empty slice

        # interpolate NaNs in each boundary column
        for j in range(b.shape[1]):
            if not np.isfinite(b[:,j]).all():
                b[:,j] = interp_nan(b[:,j])

        # normalize image to 0..255
        img_norm = img_as_ubyte((img - img.min())/(img.max()-img.min() + 1e-8))

        # save image
        base = f"{subj_name}_slice_{s:03d}"
        img_out = os.path.join(out_dir, base + "_img.png")
        Image.fromarray(img_norm).save(img_out)

        # save multi-class mask
        mask = boundaries_to_multiclass_mask(b, H)

        # --- scale for visualization (each class has unique gray intensity) ---
        mask_vis = (mask * (255 // mask.max())).astype(np.uint8)

        mask_out = os.path.join(out_dir, base + "_mask.png")
        Image.fromarray(mask_vis).save(mask_out)

        # save raw mask (true class labels, useful for training)
        np.save(os.path.join(out_dir, base + "_mask.npy"), mask.astype(np.uint8))

        # save raw boundary arrays for metrics
        np.save(os.path.join(out_dir, base + "_bounds.npy"), b.astype(np.float32))

    print(f"Processed {mat_path}")

# ---- run on all .mat files ----
mat_files = glob.glob(os.path.join(mat_dir, "*.mat"))
print(f"Found {len(mat_files)} .mat files")

for m in mat_files:
    process_mat(m, out_dir)

print("✅ All done! Multi-class processed data saved in:", out_dir)

Found 269 .mat files
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1178.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1150.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1144.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1193.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1187.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1226.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1232.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1018.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1024.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1030.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1031.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1025.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1019.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1233.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1227.mat
Processed AMD/Farsiu_Ophthalmology_2013_AMD_Subject_1186.mat
Pro

In [2]:
from PIL import Image
import numpy as np

# Load the mask
mask_path = "Multiclass_normal/Farsiu_Ophthalmology_2013_Control_Subject_1001_slice_011_mask.png"
mask = Image.open(mask_path).convert("L")   # ensure grayscale
mask_np = np.array(mask)

# Check unique pixel values
unique_vals = np.unique(mask_np)
print("Unique pixel values in mask:", unique_vals)


Unique pixel values in mask: [ 63 126 189 252]
